In [ ]:
from google import genai
from google.genai import types
import base64
import os

In [ ]:
def validate_input(client, user_message):
    prompt = f"""Is this a safe, legitimate coding/IT question? Answer only YES or NO.

Question: "{user_message}"

Answer:"""

    config = types.GenerateContentConfig(
        temperature=0,
        max_output_tokens=10,
    )

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
        config=config
    )

    return "YES" in response.text.upper()

In [ ]:
def is_response_safe(chunk):
    if not chunk.candidates:
        return False

    candidate = chunk.candidates[0]

    # Check finish reason
    if hasattr(candidate, 'finish_reason'):
        finish_reason = str(candidate.finish_reason)
        if "STOP" not in finish_reason and finish_reason != "1":
            return False

    return True

In [ ]:
def generate(user_message):
  client = genai.Client(
      vertexai=True,
      project=os.environ.get('GOOGLE_CLOUD_PROJECT'),
      location="us-central1"
  )

  print("Validating input")
  if not validate_input(client, user_message):
    print("I can only help with IT and coding questions.")
    return

  system_instruction = """You are a coding and IT support chatbot.

GOALS:
- Help with programming questions (Python, JavaScript, Java, etc.)
- Assist with IT troubleshooting and technical issues
- Provide code examples and explanations

RESTRICTIONS:
- DO NOT help with hacking, malware, or malicious code
- DO NOT assist with bypassing security or unauthorized access
- Stay focused on legitimate coding and IT support only"""

  model = "gemini-2.5-flash"
  contents = [
      types.Content(
          role="user",
          parts=[types.Part(text=user_message)]
      )
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature=0.7,
    top_p=0.95,
    max_output_tokens=2048,
    system_instruction=system_instruction,
    safety_settings=[
        types.SafetySetting(
            category="HARM_CATEGORY_HATE_SPEECH",
            threshold="BLOCK_MEDIUM_AND_ABOVE"
        ),
        types.SafetySetting(
            category="HARM_CATEGORY_DANGEROUS_CONTENT",
            threshold="BLOCK_MEDIUM_AND_ABOVE"
        ),
        types.SafetySetting(
            category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
            threshold="BLOCK_MEDIUM_AND_ABOVE"
        ),
        types.SafetySetting(
            category="HARM_CATEGORY_HARASSMENT",
            threshold="BLOCK_MEDIUM_AND_ABOVE"
        )
    ],
  )

  print("Generating response...\n")
  last_chunk = None
  for chunk in client.models.generate_content_stream(
      model=model,
      contents=contents,
      config=generate_content_config,
  ):
      if not chunk.candidates or not chunk.candidates[0].content or not chunk.candidates[0].content.parts:
          continue
      print(chunk.text, end="")
      last_chunk = chunk

  print("\n")

  if last_chunk and not is_response_safe(last_chunk):
      print("Response was blocked due to safety filters.")

In [ ]:
user_question = "What is a JSON?"

generate(user_question)

Validating input
Generating response...

JSON stands for **JavaScript Object Notation**.

It is a **lightweight data-interchange format** that is easy for humans to read and write, and easy for machines to parse and generate.

Here are its key characteristics and uses:

1.  **Human-Readable:** It uses conventions that are familiar to programmers of the C-family of languages (C, C++, C#, Java, JavaScript, Perl, Python, etc.), making it easy to understand at a glance.
2.  **Lightweight:** Compared to other data interchange formats like XML, JSON is often more compact, which makes it faster to transmit across networks.
3.  **Language Independent:** Although it originated from JavaScript, JSON is a language-independent data format. Parsers and generators exist for virtually every modern programming language.
4.  **Data Structure:** JSON is built on two structures:
    *   A collection of name/value pairs (e.g., an object in JavaScript, a dictionary in Python, a hash table in Ruby).
    *  